In [ ]:
#Convolution

In [1]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


In [2]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.
    
    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

In [3]:
import sys, os, numpy as np

In [4]:
sys.path.append(os.pardir)

In [5]:
import PIL.Image as Image
import tifffile as tiff

In [6]:
a = tiff.imread('Sentinel_sample.tif')

In [7]:
a.shape

(256, 256, 3)

In [8]:
aaa = a.reshape(1, 3, 256, 256)

In [9]:
aaa.shape

(1, 3, 256, 256)

In [10]:
aaa[0]

array([[[0.1655 , 0.2075 , 0.28125, ..., 0.21775, 0.26575, 0.2175 ],
        [0.22975, 0.278  , 0.18975, ..., 0.4    , 0.596  , 0.53125],
        [0.53475, 0.5185 , 0.46425, ..., 0.388  , 0.398  , 0.41025],
        ...,
        [0.40025, 0.4005 , 0.28275, ..., 0.27775, 0.1395 , 0.188  ],
        [0.26675, 0.18125, 0.22825, ..., 0.34425, 0.3635 , 0.4035 ],
        [0.159  , 0.1975 , 0.26175, ..., 0.48525, 0.46825, 0.38175]],

       [[0.34775, 0.36075, 0.5765 , ..., 0.274  , 0.14525, 0.19375],
        [0.2715 , 0.25625, 0.28425, ..., 0.2545 , 0.27025, 0.34275],
        [0.19775, 0.2525 , 0.3135 , ..., 0.25425, 0.29275, 0.2755 ],
        ...,
        [0.29425, 0.2375 , 0.25525, ..., 0.19975, 0.24025, 0.34925],
        [0.21825, 0.21725, 0.26525, ..., 0.20375, 0.25425, 0.14075],
        [0.208  , 0.251  , 0.147  , ..., 0.36825, 0.385  , 0.37025]],

       [[0.387  , 0.22825, 0.2335 , ..., 0.1515 , 0.1975 , 0.2975 ],
        [0.2975 , 0.30625, 0.2855 , ..., 0.22575, 0.27225, 0.18725],
    

In [11]:
x1 = np.random.rand(1,3,16,16)  #input_data 자를범위

In [12]:
col1 = im2col(x1, 8, 8, 4, 0)  #필터 5x5 stride=4, pad=0

In [13]:
print(col1.shape)

(9, 192)


In [14]:
x1.shape

(1, 3, 16, 16)

In [15]:
#9는 7x7 행렬이 5x5 행렬로 stride=1로 진행했을 때,9번 찍히기 때문, 75는 공식에 의해 5x5가 나옴, 여기에 3채널이니까 5x5x3=75임

In [16]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.col = None
        self.col_W = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        FN, FC, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad -FH) / self.stride)
        out_w = int(1 + (H + 2*self.pad -FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
        
        self.x = x
        self.col = col
        self.col_W = col_W
        return out
    
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

In [17]:
b = Convolution(x1, 0.1)

In [18]:
x1.shape

(1, 3, 16, 16)

In [19]:
bb = b.forward(aaa)

In [20]:
bb

array([[[[108.57463287, 108.77099024, 109.03004083, ..., 109.83707542,
          109.42348421, 109.88599054],
         [107.00999004, 106.91158052, 107.56931271, ..., 111.1038262 ,
          111.64904697, 111.19015569],
         [106.15594967, 106.25856979, 107.01276644, ..., 112.25869591,
          111.05203758, 111.35519702],
         ...,
         [107.58156478, 108.61653737, 110.90559968, ..., 102.18254745,
          102.85694372, 103.59359206],
         [106.30574962, 107.66354128, 108.36214217, ..., 102.9348083 ,
          102.68897654, 103.24858729],
         [105.753654  , 105.97031871, 107.46915961, ..., 102.53773712,
          102.75332393, 103.88961556]]]])

In [21]:
bb.shape

(1, 1, 241, 241)

In [22]:
#Pooling

In [23]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (H - self.pool_w) / self.stride)
        #out_h, w는 stride로 했을 때, 몇번 스캐닝할 수 있는가
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)
        
        return out

In [24]:
c = Pooling(2, 2, stride=2)

In [25]:
c

In [26]:
cc = c.forward(bb)

In [27]:
cc.shape

(1, 1, 120, 120)

In [28]:
#풀링까지 한거 tiff로 저장하는 법

In [29]:
from tifffile import imsave

In [30]:
bb.shape

(1, 1, 241, 241)

In [31]:
imsave('bbbb.tif', cc)

In [32]:
im = Image.fromarray(bb, mode='F') # float32
im.save("test3.tif", "TIF")

ValueError: Too many dimensions: 4 > 2.

In [33]:
bbb = bb.reshape(241, 241)

In [34]:
241 * 241

58081